In [ ]:
import os
import copy

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import ibicus

from tqdm import tqdm

# Bias adjustment method

Implement the bias adjustment method using the ibicus package. The bias adjustment is an additive, mean-trend preserving empirical quantile mapping, with a running window over the application period. We rely on the ibicus functionality and implement a subclass to the ibicus QuantileMapping class to include the running window over the application period, which as of ibicus v.1.1.1 is not yet supported for QuantileMapping.

In [ ]:
import attrs

from typing import Union, Optional

from ibicus.utils import RunningWindowOverYears, year
from ibicus.debias import QuantileMapping

@attrs.define(slots=False)
class QuantileMappingRunningWindowFuture(QuantileMapping):
    
    # Running window mode over years
    running_window_mode_over_years_of_cm_future: bool = attrs.field(
        default=True, validator=attrs.validators.instance_of(bool)
    )
    running_window_over_years_of_cm_future_length: int = attrs.field(
        default=9, validator=attrs.validators.instance_of(int)
    )
    running_window_over_years_of_cm_future_step_length: int = attrs.field(
        default=1, validator=attrs.validators.instance_of(int) # maybe change to 1
    )

    def __attrs_post_init__(self):
        super().__attrs_post_init__()

        if self.running_window_mode_over_years_of_cm_future:
            self.running_window_over_years_of_cm_future = RunningWindowOverYears(
                window_length_in_years=self.running_window_over_years_of_cm_future_length,
                window_step_length_in_years=self.running_window_over_years_of_cm_future_step_length,
            )
        if self.cdf_threshold is None:
            self.cdf_threshold = 1 / (
                self.running_window_length
                * self.running_window_over_years_of_cm_future_length
                + 1
            )
            
    def _apply_debiasing_steps(self, obs, cm_hist, cm_future, **kwargs):
        if self.detrending == "additive":
            delta = np.mean(cm_future) - np.mean(cm_hist)
            return self._standard_qm(cm_future - delta, obs, cm_hist) + delta
        elif self.detrending == "multiplicative":
            delta = np.mean(cm_future) / np.mean(cm_hist)
            return self._standard_qm(cm_future / delta, obs, cm_hist) * delta
        elif self.detrending == "no_detrending":
            return self._standard_qm(cm_future, obs, cm_hist)
        else:
            raise ValueError(
                "self.detrending needs to be one of ['additive', 'multiplicative', 'no_detrending']"
            )
            
    def apply_on_window(
        self,
        obs: np.ndarray,
        cm_hist: np.ndarray,
        cm_future: np.ndarray,
        time_obs: Optional[np.ndarray] = None,
        time_cm_hist: Optional[np.ndarray] = None,
        time_cm_future: Optional[np.ndarray] = None,
    ) -> np.ndarray:
        
        if np.all(np.isnan(obs)) or np.all(np.isnan(cm_hist)) or np.all(np.isnan(cm_future)):
            return np.nan
        
        if self.running_window_mode_over_years_of_cm_future:

            years_cm_future = year(time_cm_future)

            debiased_cm_future = np.empty_like(cm_future)

            # Iteration over years of cm_future to account for trends
            for (
                years_to_debias,
                years_in_window,
            ) in self.running_window_over_years_of_cm_future.use(years_cm_future):
                mask_years_in_window = RunningWindowOverYears.get_if_in_chosen_years(
                    years_cm_future, years_in_window
                )
                mask_years_to_debias = RunningWindowOverYears.get_if_in_chosen_years(
                    years_cm_future, years_to_debias
                )
                mask_years_in_window_to_debias = (
                    RunningWindowOverYears.get_if_in_chosen_years(
                        years_cm_future[mask_years_in_window], years_to_debias
                    )
                )

                debiased_cm_future[mask_years_to_debias] = self._apply_debiasing_steps(
                    obs,
                    cm_hist, 
                    cm_future[mask_years_in_window]
                )[mask_years_in_window_to_debias]

            return debiased_cm_future

        else:
            return self._apply_debiasing_steps(
                obs,
                cm_hist, 
                cm_future
            )

# Debias

In [ ]:
regions = ["NW_Amazon", "MED", "Chile", "Greece", "Bolivia", "Canada", "NWN"]

The scenarios and models are relevant for the isimip3b runs for both tree and nonetree:

In [ ]:
scenarios = ['historical', 'ssp126', 'ssp370', 'ssp585']
models = ['GFDL-ESM4', 'IPSL-CM6A-LR', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'UKESM1-0-LL']

## Tree cover

isimip3b, scenario, model, region:

In [ ]:
for region in regions:
    
    tree_cover_cm_hist = xr.open_dataset(f"data/{region}/isimp3a/obsclim/GSWP3-W5E5/period_2002_2019/tree_cover_jules-es.nc")
    tree_cover_obs = xr.open_dataset(f"data/{region}/isimp3a/obsclim/GSWP3-W5E5/period_2002_2019/tree_cover_VCF-obs.nc")
    
    # Cut to right area
    obs_latitudes = tree_cover_obs.latitude.values
    obs_longitudes = tree_cover_obs.longitude.values

    if region in ["MED", "Chile"]:
        tree_cover_cm_hist = tree_cover_cm_hist.sel(
                lat=slice(max(obs_latitudes), min(obs_latitudes)),
                lon=slice(min(obs_longitudes), max(obs_longitudes)),
            )
    
    # Extract values
    tree_cover_cm_hist_np = tree_cover_cm_hist.tree_cover.values
    tree_cover_obs_np = tree_cover_obs.tree_cover.values
    
    # Convert NaNs to np.nan (sometimes these are casted to high values)
    tree_cover_cm_hist_np[tree_cover_cm_hist_np > 1e6] = np.nan
    tree_cover_obs_np[tree_cover_obs_np > 1e6] = np.nan
    
    # Convert obs values
    tree_cover_obs_np = tree_cover_obs_np*100

    # Common nan grid
    common_nan_grid_tree_cover = np.logical_or(np.all(np.isnan(tree_cover_cm_hist_np), axis = 0), np.all(np.isnan(tree_cover_obs_np), axis = 0))
    tree_cover_cm_hist_np[:, common_nan_grid_tree_cover] = np.nan
    tree_cover_obs_np[:, common_nan_grid_tree_cover] = np.nan

    print(f"--------Bias adjusting region {region}----------\n")

    for scenario in tqdm(scenarios):
        for model in models:
            try:
                if scenario == "historical":
                    period = "period_1994_2014"
                else:
                    period = "period_2015_2099"

                # Read in values
                filename = os.path.join("data", region, "isimp3b", scenario, model, period, "tree_cover_jules-es.nc")
                tree_cover_cm_fut = xr.open_dataset(filename)
                
                # Cut to the right area
                if region in ["MED", "Chile"]:
                    tree_cover_cm_fut = tree_cover_cm_fut.sel(
                        lat=slice(max(obs_latitudes), min(obs_latitudes)),
                        lon=slice(min(obs_longitudes), max(obs_longitudes)),
                    )
                
                tree_cover_cm_fut_np = tree_cover_cm_fut.tree_cover.values
                tree_cover_cm_fut_np[tree_cover_cm_fut_np > 1e6] = np.nan

                # Get common nan mask (land-sea mask)
                nan_grid_cm_future = np.all(np.isnan(tree_cover_cm_fut_np), axis = 0)

                # Get values in cm future where values exist, but the common mask indicates nan (to infill later)
                cm_future_not_nan_but_obs_or_cm_hist = np.logical_and(np.logical_not(nan_grid_cm_future), common_nan_grid_tree_cover)

                # Map to common nan mask
                tree_cover_cm_fut_np_common_nan_grid = copy.deepcopy(tree_cover_cm_fut_np)
                tree_cover_cm_fut_np_common_nan_grid[:, common_nan_grid_tree_cover] = np.nan

                # Get time information
                time_obs = tree_cover_obs.time.values
                time_cm_hist = tree_cover_cm_hist.time.values
                time_cm_future = tree_cover_cm_fut.time.values

                # Apply bias correction
                debiaser = QuantileMappingRunningWindowFuture(mapping_type="nonparametric", detrending = "additive")
                debiased_tree_cover_cm_fut_common_nan_grid = debiaser.apply(tree_cover_obs_np, tree_cover_cm_hist_np, tree_cover_cm_fut_np_common_nan_grid,  time_obs = time_obs, time_cm_hist = time_cm_hist, time_cm_future = time_cm_future, progressbar = False, failsafe = True)

                # Reinfill values in cm_future outside of common nan grid 
                debiased_tree_cover_cm_fut = debiased_tree_cover_cm_fut_common_nan_grid
                debiased_tree_cover_cm_fut[:, cm_future_not_nan_but_obs_or_cm_hist] = tree_cover_cm_fut_np[:, cm_future_not_nan_but_obs_or_cm_hist] 

                # Map values below 0 and above 100 to min, max of obs:
                debiased_tree_cover_cm_fut[debiased_tree_cover_cm_fut < 0] = tree_cover_obs_np[~np.isnan(tree_cover_obs_np)].min()
                debiased_tree_cover_cm_fut[debiased_tree_cover_cm_fut > 100] = tree_cover_obs_np[~np.isnan(tree_cover_obs_np)].max()

                # Insert back into array
                tree_cover_cm_fut.tree_cover.values = debiased_tree_cover_cm_fut

                # Write to file
                tree_cover_cm_fut.to_netcdf(os.path.join("data", region, "isimp3b", scenario, model, period, "debiased_tree_cover_jules-es.nc"))


            except Exception as e:
                print(region, scenario, model)
                print(e)


isimip3a, obsclim or counterclim, GSWP3-W5E5:

In [ ]:
for region in regions:
    
    tree_cover_cm_hist = xr.open_dataset(f"data/{region}/isimp3a/obsclim/GSWP3-W5E5/period_2002_2019/tree_cover_jules-es.nc")
    tree_cover_obs = xr.open_dataset(f"data/{region}/isimp3a/obsclim/GSWP3-W5E5/period_2002_2019/tree_cover_VCF-obs.nc")
    
    # Cut to right area
    obs_latitudes = tree_cover_obs.latitude.values
    obs_longitudes = tree_cover_obs.longitude.values

    if region in ["MED", "Chile"]:
        tree_cover_cm_hist = tree_cover_cm_hist.sel(
                lat=slice(max(obs_latitudes), min(obs_latitudes)),
                lon=slice(min(obs_longitudes), max(obs_longitudes)),
            )
    
    # Extract values
    tree_cover_cm_hist_np = tree_cover_cm_hist.tree_cover.values
    tree_cover_obs_np = tree_cover_obs.tree_cover.values
    
    # Convert NaNs to np.nan (sometimes these are casted to high values)
    tree_cover_cm_hist_np[tree_cover_cm_hist_np > 1e6] = np.nan
    tree_cover_obs_np[tree_cover_obs_np > 1e6] = np.nan
    
    # Convert obs values
    tree_cover_obs_np = tree_cover_obs_np*100

    # Common nan grid
    common_nan_grid_tree_cover = np.logical_or(np.all(np.isnan(tree_cover_cm_hist_np), axis = 0), np.all(np.isnan(tree_cover_obs_np), axis = 0))
    tree_cover_cm_hist_np[:, common_nan_grid_tree_cover] = np.nan
    tree_cover_obs_np[:, common_nan_grid_tree_cover] = np.nan

    print(f"--------Bias adjusting region {region}----------\n")

    for clim_type in ["obsclim", "counterclim"]:
        for period in ["period_2000_2019", "period_1901_1920"]:
            
            try:
                
                # Read in values
                filename = os.path.join("data", region, "isimp3a", clim_type, "GSWP3-W5E5", period, "tree_cover_jules-es.nc")
                tree_cover_cm_fut = xr.open_dataset(filename)
                
                # Cut to the right area
                if region in ["MED", "Chile"]:
                    tree_cover_cm_fut = tree_cover_cm_fut.sel(
                        lat=slice(max(obs_latitudes), min(obs_latitudes)),
                        lon=slice(min(obs_longitudes), max(obs_longitudes)),
                    )
                
                tree_cover_cm_fut_np = tree_cover_cm_fut.tree_cover.values
                tree_cover_cm_fut_np[tree_cover_cm_fut_np > 1e6] = np.nan

                # Get common nan mask (land-sea mask)
                nan_grid_cm_future = np.all(np.isnan(tree_cover_cm_fut_np), axis = 0)

                # Get values in cm future where values exist, but the common mask indicates nan (to infill later)
                cm_future_not_nan_but_obs_or_cm_hist = np.logical_and(np.logical_not(nan_grid_cm_future), common_nan_grid_tree_cover)

                # Map to common nan mask
                tree_cover_cm_fut_np_common_nan_grid = copy.deepcopy(tree_cover_cm_fut_np)
                tree_cover_cm_fut_np_common_nan_grid[:, common_nan_grid_tree_cover] = np.nan

                # Get time information
                time_obs = tree_cover_obs.time.values
                time_cm_hist = tree_cover_cm_hist.time.values
                time_cm_future = tree_cover_cm_fut.time.values

                # Apply bias correction
                debiaser = QuantileMappingRunningWindowFuture(mapping_type="nonparametric", detrending = "additive")
                debiased_tree_cover_cm_fut_common_nan_grid = debiaser.apply(tree_cover_obs_np, tree_cover_cm_hist_np, tree_cover_cm_fut_np_common_nan_grid, time_obs = time_obs, time_cm_hist = time_cm_hist, time_cm_future = time_cm_future, failsafe = True)

                # Reinfill values in cm_future outside of common nan grid 
                debiased_tree_cover_cm_fut = debiased_tree_cover_cm_fut_common_nan_grid
                debiased_tree_cover_cm_fut[:, cm_future_not_nan_but_obs_or_cm_hist] = tree_cover_cm_fut_np[:, cm_future_not_nan_but_obs_or_cm_hist] 

                # Map values below 0 and above 100 to min, max of obs:
                debiased_tree_cover_cm_fut[debiased_tree_cover_cm_fut < 0] = tree_cover_obs_np[~np.isnan(tree_cover_obs_np)].min()
                debiased_tree_cover_cm_fut[debiased_tree_cover_cm_fut > 100] = tree_cover_obs_np[~np.isnan(tree_cover_obs_np)].max()

                # Insert back into array
                tree_cover_cm_fut.tree_cover.values = debiased_tree_cover_cm_fut

                # Write to file
                tree_cover_cm_fut.to_netcdf(os.path.join("data", region, "isimp3a", clim_type, "GSWP3-W5E5", period, "debiased_tree_cover_jules-es.nc"))

            except Exception as e:
                print(region, clim_type, period)
                print(e)

## Nonetree cover

isimip3b, scenario, model, region:

In [ ]:
for region in regions:
    
    nonetree_cover_cm_hist = xr.open_dataset(f"data/{region}/isimp3a/obsclim/GSWP3-W5E5/period_2002_2019/nonetree_cover_jules-es.nc")
    nonetree_cover_obs = xr.open_dataset(f"data/{region}/isimp3a/obsclim/GSWP3-W5E5/period_2002_2019/nontree_cover_VCF-obs.nc")
    
    # Cut to right area
    obs_latitudes = nonetree_cover_obs.latitude.values
    obs_longitudes = nonetree_cover_obs.longitude.values

    if region in ["MED", "Chile"]:
        nonetree_cover_cm_hist = nonetree_cover_cm_hist.sel(
                lat=slice(max(obs_latitudes), min(obs_latitudes)),
                lon=slice(min(obs_longitudes), max(obs_longitudes)),
            )
    
    # Extract values
    nonetree_cover_cm_hist_np = nonetree_cover_cm_hist.nonetree_cover.values
    nonetree_cover_obs_np = nonetree_cover_obs.nontree_cover.values
    
    # Convert NaNs to np.nan (sometimes these are casted to high values)
    nonetree_cover_cm_hist_np[nonetree_cover_cm_hist_np > 1e6] = np.nan
    nonetree_cover_obs_np[nonetree_cover_obs_np > 1e6] = np.nan
    
    # Convert obs values
    nonetree_cover_obs_np = nonetree_cover_obs_np*100

    # Common nan grid
    common_nan_grid_nonetree_cover = np.logical_or(np.all(np.isnan(nonetree_cover_cm_hist_np), axis = 0), np.all(np.isnan(nonetree_cover_obs_np), axis = 0))
    nonetree_cover_cm_hist_np[:, common_nan_grid_nonetree_cover] = np.nan
    nonetree_cover_obs_np[:, common_nan_grid_nonetree_cover] = np.nan

    print(f"--------Bias adjusting region {region}----------\n")

    for scenario in tqdm(scenarios):
        for model in models:
            try:
                if scenario == "historical":
                    period = "period_1994_2014"
                else:
                    period = "period_2015_2099"

                # Read in values
                filename = os.path.join("data", region, "isimp3b", scenario, model, period, "nonetree_cover_jules-es.nc")
                nonetree_cover_cm_fut = xr.open_dataset(filename)
                
                # Cut to the right area
                if region in ["MED", "Chile"]:
                    nonetree_cover_cm_fut = nonetree_cover_cm_fut.sel(
                        lat=slice(max(obs_latitudes), min(obs_latitudes)),
                        lon=slice(min(obs_longitudes), max(obs_longitudes)),
                    )
                
                nonetree_cover_cm_fut_np = nonetree_cover_cm_fut.nonetree_cover.values
                nonetree_cover_cm_fut_np[nonetree_cover_cm_fut_np > 1e6] = np.nan

                # Get common nan mask (land-sea mask)
                nan_grid_cm_future = np.all(np.isnan(nonetree_cover_cm_fut_np), axis = 0)

                # Get values in cm future where values exist, but the common mask indicates nan (to infill later)
                cm_future_not_nan_but_obs_or_cm_hist = np.logical_and(np.logical_not(nan_grid_cm_future), common_nan_grid_nonetree_cover)

                # Map to common nan mask
                nonetree_cover_cm_fut_np_common_nan_grid = copy.deepcopy(nonetree_cover_cm_fut_np)
                nonetree_cover_cm_fut_np_common_nan_grid[:, common_nan_grid_nonetree_cover] = np.nan

                # Get time information
                time_obs = nonetree_cover_obs.time.values
                time_cm_hist = nonetree_cover_cm_hist.time.values
                time_cm_future = nonetree_cover_cm_fut.time.values

                # Apply bias correction
                debiaser = QuantileMappingRunningWindowFuture(mapping_type="nonparametric", detrending = "additive")
                debiased_nonetree_cover_cm_fut_common_nan_grid = debiaser.apply(nonetree_cover_obs_np, nonetree_cover_cm_hist_np, nonetree_cover_cm_fut_np_common_nan_grid,  time_obs = time_obs, time_cm_hist = time_cm_hist, time_cm_future = time_cm_future, progressbar = False, failsafe = True)

                # Reinfill values in cm_future outside of common nan grid 
                debiased_nonetree_cover_cm_fut = debiased_nonetree_cover_cm_fut_common_nan_grid
                debiased_nonetree_cover_cm_fut[:, cm_future_not_nan_but_obs_or_cm_hist] = nonetree_cover_cm_fut_np[:, cm_future_not_nan_but_obs_or_cm_hist] 

                # Map values below 0 and above 100 to min, max of obs:
                debiased_nonetree_cover_cm_fut[debiased_nonetree_cover_cm_fut < 0] = nonetree_cover_obs_np[~np.isnan(nonetree_cover_obs_np)].min()
                debiased_nonetree_cover_cm_fut[debiased_nonetree_cover_cm_fut > 100] = nonetree_cover_obs_np[~np.isnan(nonetree_cover_obs_np)].max()

                # Insert back into array
                nonetree_cover_cm_fut.nonetree_cover.values = debiased_nonetree_cover_cm_fut

                # Write to file
                nonetree_cover_cm_fut.to_netcdf(os.path.join("data", region, "isimp3b", scenario, model, period, "debiased_nonetree_cover_jules-es.nc"))


            except Exception as e:
                print(region, scenario, model)
                print(e)

isimip3a, obsclim or counterclim, GSWP3-W5E5:

In [ ]:
for region in regions:
    
    nonetree_cover_cm_hist = xr.open_dataset(f"data/{region}/isimp3a/obsclim/GSWP3-W5E5/period_2002_2019/nonetree_cover_jules-es.nc")
    nonetree_cover_obs = xr.open_dataset(f"data/{region}/isimp3a/obsclim/GSWP3-W5E5/period_2002_2019/nontree_cover_VCF-obs.nc")
    
    # Cut to right area
    obs_latitudes = nonetree_cover_obs.latitude.values
    obs_longitudes = nonetree_cover_obs.longitude.values

    if region in ["MED", "Chile"]:
        nonetree_cover_cm_hist = nonetree_cover_cm_hist.sel(
                lat=slice(max(obs_latitudes), min(obs_latitudes)),
                lon=slice(min(obs_longitudes), max(obs_longitudes)),
            )
    
    # Extract values
    nonetree_cover_cm_hist_np = nonetree_cover_cm_hist.nonetree_cover.values
    nonetree_cover_obs_np = nonetree_cover_obs.nontree_cover.values
    
    # Convert NaNs to np.nan (sometimes these are casted to high values)
    nonetree_cover_cm_hist_np[nonetree_cover_cm_hist_np > 1e6] = np.nan
    nonetree_cover_obs_np[nonetree_cover_obs_np > 1e6] = np.nan
    
    # Convert obs values
    nonetree_cover_obs_np = nonetree_cover_obs_np*100

    # Common nan grid
    common_nan_grid_nonetree_cover = np.logical_or(np.all(np.isnan(nonetree_cover_cm_hist_np), axis = 0), np.all(np.isnan(nonetree_cover_obs_np), axis = 0))
    nonetree_cover_cm_hist_np[:, common_nan_grid_nonetree_cover] = np.nan
    nonetree_cover_obs_np[:, common_nan_grid_nonetree_cover] = np.nan

    print(f"--------Bias adjusting region {region}----------\n")

    for clim_type in ["obsclim", "counterclim"]:
        for period in ["period_2000_2019", "period_1901_1920"]:
            
            try:
                
                # Read in values
                filename = os.path.join("data", region, "isimp3a", clim_type, "GSWP3-W5E5", period, "nonetree_cover_jules-es.nc")
                nonetree_cover_cm_fut = xr.open_dataset(filename)
                
                # Cut to the right area
                if region in ["MED", "Chile"]:
                    nonetree_cover_cm_fut = nonetree_cover_cm_fut.sel(
                        lat=slice(max(obs_latitudes), min(obs_latitudes)),
                        lon=slice(min(obs_longitudes), max(obs_longitudes)),
                    )
                
                nonetree_cover_cm_fut_np = nonetree_cover_cm_fut.nonetree_cover.values
                nonetree_cover_cm_fut_np[nonetree_cover_cm_fut_np > 1e6] = np.nan

                # Get common nan mask (land-sea mask)
                nan_grid_cm_future = np.all(np.isnan(nonetree_cover_cm_fut_np), axis = 0)

                # Get values in cm future where values exist, but the common mask indicates nan (to infill later)
                cm_future_not_nan_but_obs_or_cm_hist = np.logical_and(np.logical_not(nan_grid_cm_future), common_nan_grid_nonetree_cover)

                # Map to common nan mask
                nonetree_cover_cm_fut_np_common_nan_grid = copy.deepcopy(nonetree_cover_cm_fut_np)
                nonetree_cover_cm_fut_np_common_nan_grid[:, common_nan_grid_nonetree_cover] = np.nan

                # Get time information
                time_obs = nonetree_cover_obs.time.values
                time_cm_hist = nonetree_cover_cm_hist.time.values
                time_cm_future = nonetree_cover_cm_fut.time.values

                # Apply bias correction
                debiaser = QuantileMappingRunningWindowFuture(mapping_type="nonparametric", detrending = "additive")
                debiased_nonetree_cover_cm_fut_common_nan_grid = debiaser.apply(nonetree_cover_obs_np, nonetree_cover_cm_hist_np, nonetree_cover_cm_fut_np_common_nan_grid, time_obs = time_obs, time_cm_hist = time_cm_hist, time_cm_future = time_cm_future, failsafe = True)

                # Reinfill values in cm_future outside of common nan grid 
                debiased_nonetree_cover_cm_fut = debiased_nonetree_cover_cm_fut_common_nan_grid
                debiased_nonetree_cover_cm_fut[:, cm_future_not_nan_but_obs_or_cm_hist] = nonetree_cover_cm_fut_np[:, cm_future_not_nan_but_obs_or_cm_hist] 

                # Map values below 0 and above 100 to min, max of obs:
                debiased_nonetree_cover_cm_fut[debiased_nonetree_cover_cm_fut < 0] = nonetree_cover_obs_np[~np.isnan(nonetree_cover_obs_np)].min()
                debiased_nonetree_cover_cm_fut[debiased_nonetree_cover_cm_fut > 100] = nonetree_cover_obs_np[~np.isnan(nonetree_cover_obs_np)].max()

                # Insert back into array
                nonetree_cover_cm_fut.nonetree_cover.values = debiased_nonetree_cover_cm_fut

                # Write to file
                nonetree_cover_cm_fut.to_netcdf(os.path.join("data", region, "isimp3a", clim_type, "GSWP3-W5E5", period, "debiased_nonetree_cover_jules-es.nc"))

            except Exception as e:
                print(region, clim_type, period)
                print(e)